# HW5


In [2]:
library(magrittr)
library(dplyr)
options(warn=-1)
setwd("C:/Users/Deren/Desktop/math388e")

Warning message:
"package 'magrittr' was built under R version 3.5.1"Warning message:
"package 'dplyr' was built under R version 3.5.1"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In the following code, I've collected the data for each person in a row for each variable (RSS_anchor1, RSS_anchor2, RSS_anchor3, RSS_anchor4) and then I  joined this data frame  with MovementAAL_target.csv data by sequence id. Our problem at this point is that the number of values in the each row are different because the number of data each person is different. In order to implement the SVM , we need to fill the rows with NA to equalize the number of values in rows.

In [3]:
data <- tibble::tribble(~X.RSS_anchor1, ~RSS_anchor2, ~RSS_anchor3, ~RSS_anchor4)
n=314

for(i in 1:n){
  

file_name <- paste0("data/rss_data/dataset/MovementAAL_RSS_", i) 
  
df <- read.csv(paste0(file_name,".csv"))
    
df %<>% add_row(X.RSS_anchor1=list(df[,1]), 
                RSS_anchor2=list(df[,2]),
                RSS_anchor3=list(df[,3]),
                RSS_anchor4=list(df[,4])) 

df1 <- df[nrow(df),]

data %<>% add_row(X.RSS_anchor1=df1[1,1],
               RSS_anchor2=df1[1,2],
               RSS_anchor3=df1[1,3],
               RSS_anchor4=df1[1,4]) 

}

data %<>% rename(RSS_anchor1=X.RSS_anchor1) %>% mutate(sequence_ID=c(1:314)) %>% select(sequence_ID,everything())

target_data <- read.csv("data/rss_data/dataset/MovementAAL_target.csv") %>% rename(sequence_ID=X.sequence_ID)

data <- left_join(data,target_data, by="sequence_ID") %>% .[,-1]

In [4]:
data %>% head(5)

RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4,class_label
"-0.904760, -0.571430, -0.380950, -0.285710, -0.142860, -0.142860, -0.142860, -0.142860, -0.095238, -0.047619, -0.190480, -0.095238, -0.333330, -0.238100, 0.000000, -0.095238, -0.142860, -0.190480, -0.142860, -0.333330, -0.428570, -0.714290, -0.095238, -0.285710, 0.000000, 0.000000, -0.142860","-0.48, -0.32, -0.28, -0.20, -0.20, -0.20, -0.16, -0.04, -0.08, 0.04, -0.04, -0.04, -0.08, 0.04, 0.08, 0.04, -0.20, 0.04, -0.08, 0.16, 0.16, 0.16, -0.08, 0.04, 0.04, 0.04, -0.60","0.285710, 0.142860, -0.142860, -0.476190, 0.142860, 0.047619, -0.380950, -0.619050, 0.142860, -0.095238, 0.095238, -0.142860, -0.285710, 0.142860, 0.142860, 0.095238, 0.142860, -0.428570, -0.238100, -0.142860, -0.285710, -0.285710, 0.095238, 0.142860, 0.142860, -0.047619, -0.285710","0.30, 0.30, 0.35, 0.35, -0.20, 0.00, 0.20, -0.20, -0.55, 0.05, 0.40, 0.35, -0.20, 0.35, 0.05, 0.10, 0.50, 0.30, 0.15, -0.80, -0.10, 0.20, 0.35, 0.20, 0.10, -0.05, -0.10",1
"-0.57143, -0.76190, -0.85714, -0.76190, -0.76190, -0.76190, -0.80952, -0.80952, -0.66667, -0.66667, -0.61905, -0.80952, -0.80952, -0.76190, -0.52381, -0.33333, -0.28571, -0.28571, -0.28571, -0.33333, -0.28571, -0.19048, -0.19048, -0.19048, -0.33333, -0.14286","-0.20, -0.48, -0.60, -0.40, -0.84, -0.84, -0.88, -0.88, -0.52, -0.52, -0.40, -0.32, -0.48, -0.60, -0.64, -0.36, -0.20, -0.08, -0.08, -0.16, -0.64, -0.52, -0.20, -0.08, 0.04, 0.04","0.714290, 0.761900, 0.857140, 0.714290, 0.857140, 0.714290, 0.761900, 0.714290, 0.761900, 0.761900, 0.380950, 0.142860, 0.285710, 0.000000, -0.142860, 0.047619, -0.095238, -0.428570, 0.047619, 0.142860, 0.000000, 0.000000, 0.190480, -0.333330, 0.095238, 0.142860","0.50, -0.25, 0.55, 0.60, 0.45, 0.05, 0.65, 0.65, 0.65, 0.65, 0.65, 0.80, 0.80, 0.80, 0.65, 0.65, 0.55, 0.35, 0.40, -0.05, 0.35, 0.05, 0.25, -0.10, -0.10, 0.05",1
"-1.00000, -0.33333, -0.76190, -0.80952, -0.85714, -0.85714, -0.85714, -0.85714, -0.85714, -1.00000, -1.00000, -0.95238, -0.57143, -0.57143, -0.71429, -0.57143, -0.42857, -0.38095, -0.42857, -0.57143, -0.42857, -0.28571, -0.14286","-0.12, -0.08, -0.84, -0.64, -0.24, -0.28, -0.44, -0.44, -0.32, -0.56, -0.60, -0.40, -0.44, -0.32, -0.20, -0.20, -0.40, -0.20, -0.20, -0.32, -0.20, -0.04, -0.08","0.571430, 0.666670, 0.761900, 0.714290, 0.714290, 0.761900, 0.714290, 0.761900, 0.857140, 0.857140, 0.857140, 0.857140, 0.714290, 0.571430, 0.000000, 0.142860, 0.333330, 0.285710, 0.047619, 0.142860, -0.142860, -0.095238, 0.142860","0.65, 0.65, 0.50, -0.30, 0.75, 0.40, 0.35, 0.55, -0.25, 0.65, 0.35, 0.80, 0.80, 0.80, 0.75, 0.75, 0.45, 0.40, 0.40, 0.35, 0.30, -0.05, 0.35",1
"-1.00000, -0.80952, -1.00000, -0.66667, -1.00000, -0.80952, -1.00000, -0.71429, -1.00000, -0.71429, -0.61905, -0.85714, -0.66667, -0.66667, -0.57143, -0.61905, -1.00000, -0.42857, -0.42857, -0.42857, -0.57143, -0.42857, -0.57143","-0.84, -0.40, -0.56, -0.16, -0.84, -0.40, -0.36, -0.52, -0.20, -0.60, -0.76, -0.64, -0.56, -0.20, -0.32, -0.32, -0.32, -0.56, -0.52, -0.32, -0.08, -0.20, -0.52","0.571430, 0.190480, 0.476190, 0.714290, 0.714290, 0.809520, 0.714290, 0.857140, 0.761900, 0.857140, 0.857140, 0.714290, 0.714290, 0.714290, 0.857140, 0.857140, 0.666670, -0.095238, 0.047619, 0.285710, 0.238100, 0.000000, 0.047619","0.35, 0.65, 0.35, 0.80, 0.55, 0.65, 0.35, -0.35, 0.75, 0.80, 0.55, -0.30, 0.20, 0.55, 0.80, 0.80, 0.65, 0.65, 0.50, 0.25, 0.25, 0.40, -0.15",1
"-1.00000, -0.52381, -0.57143, -0.66667, -0.90476, -0.47619, -1.00000, -0.71429, -0.71429, -0.71429, -0.47619, -1.00000, -1.00000, -0.95238, -0.66667, -1.00000, -0.95238, -1.00000, -1.00000, -1.00000, -0.76190, -1.00000, -0.90476, -0.57143, -0.52381","-0.28, -0.32, -0.32, -0.40, -0.56, -0.44, -0.44, -0.48, -0.48, -0.56, -0.16, -0.56, -0.32, -0.48, -0.52, -0.56, -0.68, -0.64, -0.72, -0.56, -0.48, -0.80, -0.60, -0.60, -0.44","0.33333, 0.52381, 0.71429, 0.57143, 0.52381, 0.57143, 0.57143, 0.42857, 0.19048, 0.33333, 0.33333, 0.80952, 0.71429, 0.71429, 0.85714, 0.71429, 0.95238, 0.80952, 0.76190, 

 I took each column as a vector element and added "NA" to the each element of the vector until the length of the longest element. As a result, each element of the vector had the same number of values and SVM model became applicable.

In [6]:
for(i in 1:4){
  
  column_name <- paste0("RSS_anchor",i)
  x <- pull(data,column_name )
  
  x %>% lapply(length) %>% unlist() %>% max -> mymax
  x %<>% lapply(function(y) c(y,rep(NA,mymax-length(y))))
  
  m <- matrix(x, ncol=1, byrow=TRUE)
  d <- as.data.frame(m, stringsAsFactors=FALSE)
  names(d)[1]<-column_name
  d[,1] <-  vapply(d[,1],paste, collapse = ", ", character(1L))


  if(i==1) {
    df <-d
  }else{
    df %<>% cbind(d)
  }
}
                
df %<>%cbind(target_data)  %>% select(-sequence_ID)
                


In [7]:
df %>% head(5)

RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4,class_label
"-0.90476, -0.57143, -0.38095, -0.28571, -0.14286, -0.14286, -0.14286, -0.14286, -0.095238, -0.047619, -0.19048, -0.095238, -0.33333, -0.2381, 0, -0.095238, -0.14286, -0.19048, -0.14286, -0.33333, -0.42857, -0.71429, -0.095238, -0.28571, 0, 0, -0.14286, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA","-0.48, -0.32, -0.28, -0.2, -0.2, -0.2, -0.16, -0.04, -0.08, 0.04, -0.04, -0.04, -0.08, 0.04, 0.08, 0.04, -0.2, 0.04, -0.08, 0.16, 0.16, 0.16, -0.08, 0.04, 0.04, 0.04, -0.6, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA","0.28571, 0.14286, -0.14286, -0.47619, 0.14286, 0.047619, -0.38095, -0.61905, 0.14286, -0.095238, 0.095238, -0.14286, -0.28571, 0.14286, 0.14286, 0.095238, 0.14286, -0.42857, -0.2381, -0.14286, -0.28571, -0.28571, 0.095238, 0.14286, 0.14286, -0.047619, -0.28571, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA","0.3, 0.3, 0.35, 0.35, -0.2, 0, 0.2, -0.2, -0.55, 0.05, 0.4, 0.35, -0.2, 0.35, 0.05, 0.1, 0.5, 0.3, 0.15, -0.8, -0.1, 0.2, 0.35, 0.2, 0.1, -0.05, -0.1, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA",1
"-0.57143, -0.7619, -0.85714, -0.7619, -0.7619, -0.7619, -0.80952, -0.80952, -0.66667, -0.66667, -0.61905, -0.80952, -0.80952, -0.7619, -0.52381, -0.33333, -0.28571, -0.28571, -0.28571, -0.33333, -0.28571, -0.19048, -0.19048, -0.19048, -0.33333, -0.14286, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA","-0.2, -0.48, -0.6, -0.4, -0.84, -0.84, -0.88, -0.88, -0.52, -0.52, -0.4, -0.32, -0.48, -0.6, -0.64, -0.36, -0.2, -0.08, -0.08, -0.16, -0.64, -0.52, -0.2, -0.08, 0.04, 0.04, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA","0.71429, 0.7619, 0.85714, 0.71429, 0.85714, 0.71429, 0.7619, 0.71429, 0.7619, 0.7619, 0.38095, 0.14286, 0.28571, 0, -0.14286, 0.047619, -0.095238, -0.42857, 0.047619, 0.14286, 0, 0, 0.19048, -0.33333, 0.09523

The following code, I used  10-fold cross-validation to estimate accuracy. This split our data set into 10 parts,  train in 9 and test on 1 and repeat all combinations of train-test split. I used the metric of "Accuracy" to evalute model. This is a ratio of the number of correctly predicted instances divided by the total number of instances in the dataset multiplied by 100 to give a percentage.

In [8]:
library(caret)

df[["class_label"]] = factor(df[["class_label"]])

trctrl <- trainControl(method = "repeatedcv", number = 10, repeats = 5)
  
svm_model <- train(class_label ~., data = df,
                   method = "svmLinear", 
                   trControl=trctrl,
                   metric="Accuracy",
                   preProcess = c("center", "scale"))

svm_model

Loading required package: lattice
Loading required package: ggplot2


Support Vector Machines with Linear Kernel 

314 samples
  4 predictor
  2 classes: '-1', '1' 

Pre-processing: centered (1252), scaled (1252) 
Resampling: Cross-Validated (10 fold, repeated 5 times) 
Summary of sample sizes: 282, 283, 284, 282, 283, 282, ... 
Resampling results:

  Accuracy   Kappa
  0.5032258  0    

Tuning parameter 'C' was held constant at a value of 1